# Utilizando GeoPy com Bing

In [1]:
from geopy.geocoders import Bing

In [5]:
# aqui cada um deve gerar uma chave para acesso a API
from secrets.bing_key import bing_key

In [7]:
geolocator = Bing(api_key=bing_key(), timeout=10, user_agent="primeiro_teste_com_Bing")

In [14]:
location = geolocator.geocode("Rua São Bento, 407, São Paulo, São Paulo, SP, BR", culture='BR')

In [15]:
location

Location(Rua São Bento, 407, Sao Paulo, SP 01010-001, Brazil, (-23.5455714, -46.6349651, 0.0))

In [16]:
import pandas as pd

In [17]:
df = pd.read_csv(f"dados/exemplo-1.csv")

In [40]:
df['endereco_sao_paulo'] = df['endereço'] +  ', São Paulo, São Paulo, SP, BR'

In [41]:
df['endereco_sao_paulo']

0                    TESTE, São Paulo, São Paulo, SP, BR
1       rua da entidade,25, São Paulo, São Paulo, SP, BR
2      Av. Augusto Antunes 337, São Paulo, São Paulo,...
3      Ermelino Matarazzo e comunidade vila Silvia, S...
4      Rua Serra de LUIZ Gomes 1992 cep 08190 500, Sã...
                             ...                        
778    Rua Maria Amália Lopes de Azevedo, 4.180. Jd. ...
779    Projeto fazendo a diferença.. Centro de São Pa...
780        RUA ITAPIRU 137, São Paulo, São Paulo, SP, BR
781      Rua do outono 410, São Paulo, São Paulo, SP, BR
782    Rua 26 de abril 117 Na sede ou no espaço socia...
Name: endereco_sao_paulo, Length: 783, dtype: object

In [19]:
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0.1)

In [20]:
from tqdm import tqdm
tqdm.pandas()

/home/fernando/miniconda3/envs/geo_env/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [42]:
df['location'] = df['endereco_sao_paulo'].progress_apply(geocode)

100%|██████████| 783/783 [13:27<00:00,  1.03s/it]


In [43]:
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [44]:
df[~df.point.isin([None])]

,id,endereço,endereco_sao_paulo,location,point
0,1,TESTE,"TESTE, São Paulo, São Paulo, SP, BR","(Rua Trieste, Sao Paulo, SP 03581-180, Brazil,...","(-23.55696, -46.49914, 0.0)"
1,2,"rua da entidade,25","rua da entidade,25, São Paulo, São Paulo, SP, BR","(Rua Vinte e Cinco, Sao Paulo, SP 02814-030, B...","(-23.4593, -46.70898, 0.0)"
2,3,Av. Augusto Antunes 337,"Av. Augusto Antunes 337, São Paulo, São Paulo,...","(Avenida Augusto Antunes, 337, Sao Paulo, SP 0...","(-23.51056, -46.46829, 0.0)"
3,5,Ermelino Matarazzo e comunidade vila Silvia,"Ermelino Matarazzo e comunidade vila Silvia, S...","(São Paulo, SP, Brazil, (-23.493900299072266, ...","(-23.493900299072266, -46.50382995605469, 0.0)"
4,6,Rua Serra de LUIZ Gomes 1992 cep 08190 500,"Rua Serra de LUIZ Gomes 1992 cep 08190 500, Sã...","(Rua Serra de Luís Gomes, 1992, Sao Paulo, SP ...","(-23.4834617, -46.3915417, 0.0)"
...,...,...,...,...,...
778,855,"Rua Maria Amália Lopes de Azevedo, 4.180. Jd. ...","Rua Maria Amália Lopes de Azevedo, 4.180. Jd. ...","(Rua Maria Amália Lopes de Azevedo, Sao Paulo,...","(-23.46065, -46.62237, 0.0)"
779,856,Projeto fazendo a diferença.. Centro de São Pa...,Projeto fazendo a diferença.. Centro de São Pa...,"(São Paulo, SP, Brazil, (-23.562870025634766, ...","(-23.562870025634766, -46.65467834472656, 0.0)"
780,857,RUA ITAPIRU 137,"RUA ITAPIRU 137, São Paulo, São Paulo, SP, BR","(Rua Itapiru, 137, Sao Paulo, SP 04143-010, Br...","(-23.61346, -46.63664, 0.0)"
781,858,Rua do outono 410,"Rua do outono 410, São Paulo, São Paulo, SP, BR","(Rua do Outono, 410, Sao Paulo, SP 02847-060, ...","(-23.46071, -46.69478, 0.0)"


In [45]:
df.to_csv("resultados/exemplo-1-bing.csv")

In [46]:
import geopandas as gpd

In [47]:
df_points = df[~df.point.isin([None])]

In [48]:
df_points['x'] = df_points.apply(lambda row: row.point[1], axis=1)
df_points['y'] = df_points.apply(lambda row: row.point[0], axis=1)

<ipython-input-48-a89875fe4032>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_points['x'] = df_points.apply(lambda row: row.point[1], axis=1)
<ipython-input-48-a89875fe4032>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_points['y'] = df_points.apply(lambda row: row.point[0], axis=1)


In [49]:
gdf = gpd.GeoDataFrame(df_points, geometry=gpd.points_from_xy(df_points.x, df_points.y))

In [50]:
gdf

,id,endereço,endereco_sao_paulo,location,point,x,y,geometry
0,1,TESTE,"TESTE, São Paulo, São Paulo, SP, BR","(Rua Trieste, Sao Paulo, SP 03581-180, Brazil,...","(-23.55696, -46.49914, 0.0)",-46.499140,-23.556960,POINT (-46.49914 -23.55696)
1,2,"rua da entidade,25","rua da entidade,25, São Paulo, São Paulo, SP, BR","(Rua Vinte e Cinco, Sao Paulo, SP 02814-030, B...","(-23.4593, -46.70898, 0.0)",-46.708980,-23.459300,POINT (-46.70898 -23.45930)
2,3,Av. Augusto Antunes 337,"Av. Augusto Antunes 337, São Paulo, São Paulo,...","(Avenida Augusto Antunes, 337, Sao Paulo, SP 0...","(-23.51056, -46.46829, 0.0)",-46.468290,-23.510560,POINT (-46.46829 -23.51056)
3,5,Ermelino Matarazzo e comunidade vila Silvia,"Ermelino Matarazzo e comunidade vila Silvia, S...","(São Paulo, SP, Brazil, (-23.493900299072266, ...","(-23.493900299072266, -46.50382995605469, 0.0)",-46.503830,-23.493900,POINT (-46.50383 -23.49390)
4,6,Rua Serra de LUIZ Gomes 1992 cep 08190 500,"Rua Serra de LUIZ Gomes 1992 cep 08190 500, Sã...","(Rua Serra de Luís Gomes, 1992, Sao Paulo, SP ...","(-23.4834617, -46.3915417, 0.0)",-46.391542,-23.483462,POINT (-46.39154 -23.48346)
...,...,...,...,...,...,...,...,...
778,855,"Rua Maria Amália Lopes de Azevedo, 4.180. Jd. ...","Rua Maria Amália Lopes de Azevedo, 4.180. Jd. ...","(Rua Maria Amália Lopes de Azevedo, Sao Paulo,...","(-23.46065, -46.62237, 0.0)",-46.622370,-23.460650,POINT (-46.62237 -23.46065)
779,856,Projeto fazendo a diferença.. Centro de São Pa...,Projeto fazendo a diferença.. Centro de São Pa...,"(São Paulo, SP, Brazil, (-23.562870025634766, ...","(-23.562870025634766, -46.65467834472656, 0.0)",-46.654678,-23.562870,POINT (-46.65468 -23.56287)
780,857,RUA ITAPIRU 137,"RUA ITAPIRU 137, São Paulo, São Paulo, SP, BR","(Rua Itapiru, 137, Sao Paulo, SP 04143-010, Br...","(-23.61346, -46.63664, 0.0)",-46.636640,-23.613460,POINT (-46.63664 -23.61346)
781,858,Rua do outono 410,"Rua do outono 410, São Paulo, São Paulo, SP, BR","(Rua do Outono, 410, Sao Paulo, SP 02847-060, ...","(-23.46071, -46.69478, 0.0)",-46.694780,-23.460710,POINT (-46.69478 -23.46071)


In [51]:
gdf[['id', 'endereço', 'geometry']].to_file('resultados/exemplo-1-bing.gpkg', driver="GPKG")